In [1]:
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.layers import Activation, Dense, Dropout, Flatten, InputLayer
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray
from skimage.io import imsave
import numpy as np
import os
import random
import tensorflow as tf

Using TensorFlow backend.


In [9]:
# Get images
X = []
list_filename = os.listdir('../images/Train/')
for filename in list_filename[0:100]:
    X.append(img_to_array(load_img('../images/Train/'+filename)))
X = np.array(X, dtype=float)

# Set up train and test data
split = int(0.95*len(X))
Xtrain = X[:split]
Xtrain = 1.0/255*Xtrain

In [10]:
model = Sequential()
model.add(InputLayer(input_shape=(256, 256, 1)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', strides=2))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
model.add(UpSampling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
model.add(Conv2D(2, (3, 3), activation='tanh', padding='same'))
model.add(UpSampling2D((2, 2)))
model.compile(optimizer='rmsprop', loss='mse')

In [ ]:
# Image transformer
datagen = ImageDataGenerator(shear_range=0.2, zoom_range=0.2, rotation_range=20, horizontal_flip=True)

# Generate training data
batch_size = 10
def image_a_b_gen(batch_size):
    for batch in datagen.flow(Xtrain, batch_size=batch_size):
        lab_batch = rgb2lab(batch)
        X_batch = lab_batch[:,:,:,0]
        Y_batch = lab_batch[:,:,:,1:] / 128
        yield (X_batch.reshape(X_batch.shape+(1,)), Y_batch)

# Train model      
tensorboard = TensorBoard(log_dir="output/first_run")
model.fit_generator(image_a_b_gen(batch_size), callbacks=[tensorboard], epochs=1, steps_per_epoch=10)

In [19]:
# Save model
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [21]:
# Test images
Xtest = rgb2lab(1.0/255*X[split:])[:,:,:,0]
Xtest = Xtest.reshape(Xtest.shape+(1,))
Ytest = rgb2lab(1.0/255*X[split:])[:,:,:,1:]
Ytest = Ytest / 128
print(model.evaluate(Xtest, Ytest, batch_size=batch_size))

5/5 [==============================] - 0s 81ms/step
0.016548306867480277


In [25]:
color_me = []
for filename in os.listdir('../testdata/Validate/'):
    color_me.append(img_to_array(load_img('../testdata/Validate/'+filename)))
color_me = np.array(color_me, dtype=float)
color_me = rgb2lab(1.0/255*color_me)[:,:,:,0]
color_me = color_me.reshape(color_me.shape+(1,))

# Test model
output = model.predict(color_me)
output = output * 128

# Output colorizations
for i in range(len(output)):
    cur = np.zeros((256, 256, 3))
    cur[:,:,0] = color_me[i][:,:,0]
    cur[:,:,1:] = output[i]
    imsave("result/img_"+str(i)+".png", lab2rgb(cur))

C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\skimage\color\colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 48653 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
W0623 10:08:57.172533 22664 util.py:58] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\skimage\color\colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 62136 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
W0623 10:08:57.407903 22664 util.py:58] Lossy conversion from float64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
C:\Users\RUSHI\Anaconda Python\envs\tf-gpu\lib\site-packages\skimage\color\colorconv.py:1068: UserWarning: Color data out of range: Z < 0 in 44046 pixels
  return xyz2rgb(lab2xyz(lab, illuminant, observer))
W0623 10:08:57.491679 22664 util.py:58] Lossy conversion from

In [29]:
lab2rgb(cur)

array([[[1.        , 0.54976116, 0.47629338],
        [1.        , 0.55431219, 0.47985127],
        [1.        , 0.47694176, 0.27988346],
        ...,
        [1.        , 0.56659113, 0.44284055],
        [1.        , 0.75364566, 0.64715581],
        [1.        , 0.74969469, 0.6434793 ]],

       [[1.        , 0.54976116, 0.47629338],
        [1.        , 0.54976116, 0.47629338],
        [1.        , 0.47190072, 0.27605908],
        ...,
        [1.        , 0.56848777, 0.44435745],
        [1.        , 0.75927547, 0.65239737],
        [1.        , 0.76549275, 0.65818972]],

       [[1.        , 0.56843092, 0.56905042],
        [1.        , 0.56843092, 0.56905042],
        [1.        , 0.55744836, 0.34648105],
        ...,
        [1.        , 0.6523716 , 0.34567629],
        [1.        , 0.81152418, 0.46739006],
        [1.        , 0.80764187, 0.46373453]],

       ...,

       [[1.        , 0.70599056, 0.65384419],
        [1.        , 0.70198976, 0.65015808],
        [1.        , 0